Prompt:

Schreibe einen Python Code, der folgende Funktionen erfüllt:
- Nutzen der CSV mit den Embeddings
- Funktion bauen um eine Frage als text zu embedden
- der embeddete text soll mit den anderen embeddings informationen nach simularity verglichen werden
- es sollen die top 3 nährstens cusens ausgegeben werden

In [1]:
import openai
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
import sys
import os

path_csv_Data = './embeddings.csv'
df = pd.read_csv(path_csv_Data, error_bad_lines=False)
df['embedding'] = df['embedding'].apply(eval).apply(np.array)

openai.api_key = os.getenv("OPENAI_API_KEY")
model = "gpt-3.5-turbo"

def get_response(message, df, model):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": message}
    ]

    df["similarities"] = df['embedding'].apply(lambda x: cosine_similarity(x, get_embedding(message, engine="text-embedding-ada-002")))
    top_3_answers = df.sort_values("similarities", ascending=False).iloc[:3]['Sentence'].tolist()

    print('TOP 3 Antworten')
    for i, answer in enumerate(top_3_answers):
        print(f"{i + 1}: {answer}")

    prompt = "\n".join([f"{i + 1}: {answer}" for i, answer in enumerate(top_3_answers)])
    print("Prompt: " + prompt)

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages + [{"role": "assistant", "content": prompt}]
    )

    response_text = response["choices"][0]["message"]["content"]

    return response_text


/var/folders/nt/sgwrtchn5v782mbccyc213hw0000gn/T/ipykernel_28328/1683236551.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(path_csv_Data, error_bad_lines=False)


In [2]:


user_query = "Seid wann gibt es die Codersunlimited?"
print("Frage: " + user_query)

response = get_response(user_query, df, model)
print(response)


Frage: Seid wann gibt es die Codersunlimited?
TOP 3 Antworten
1: Es ist gerade einmal drei Jahre her, dass Thorsten, Uwe und ich Anfang 2020 die Xengoo Consulting mit der Coders Unlimited zu einem neuartigen MarTech Unternehmen verschmolzen haben
2: Das Management von RYZE Digital hatte 2021 den Markt nach einem Player im Segment Marketing- und Sales Automatisierung zwecks Akquisition gescannt und war so auf Coders Unlimited aufmerksam geworden
3: Liebe Kunden, Partner und Freunde von CU, es gibt großartige Neuigkeiten in eigener Sache zum Jahresauftakt
Prompt: 1: Es ist gerade einmal drei Jahre her, dass Thorsten, Uwe und ich Anfang 2020 die Xengoo Consulting mit der Coders Unlimited zu einem neuartigen MarTech Unternehmen verschmolzen haben
2: Das Management von RYZE Digital hatte 2021 den Markt nach einem Player im Segment Marketing- und Sales Automatisierung zwecks Akquisition gescannt und war so auf Coders Unlimited aufmerksam geworden
3: Liebe Kunden, Partner und Freunde von CU, 